In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import yfinance as yf
import talib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

Test LSTM using minute data

In [2]:
# data = yf.download('BTC-USD', period='2y')
data = pd.read_csv('./processed_data/BTC_1m_data_indicators.csv')
#data = pd.read_csv('./market_data/BTC_1m_data.csv')

In [3]:
# data.index = pd.to_datetime(data.index)
data.set_index('Open Time', inplace = True)


In [4]:
""" macd_indicator = ta.trend.MACD(close=data['Close'].squeeze(), window_slow=26, window_fast=12, window_sign=9)
data['MACD'] = macd_indicator.macd()  # MACD Line
data['MACD_signal'] = macd_indicator.macd_signal()  # Signal Line
data['MACD_hist'] = macd_indicator.macd_diff()  # Histogram (MACD - Signal) """


" macd_indicator = ta.trend.MACD(close=data['Close'].squeeze(), window_slow=26, window_fast=12, window_sign=9)\ndata['MACD'] = macd_indicator.macd()  # MACD Line\ndata['MACD_signal'] = macd_indicator.macd_signal()  # Signal Line\ndata['MACD_hist'] = macd_indicator.macd_diff()  # Histogram (MACD - Signal) "

In [5]:
data = data['2025-01-01':]

In [6]:
data

,Open,High,Low,Close,Volume,Number of Trades,Coin,EMA_9,EMA_21,MACD,MACD_Signal,MACD_Hist,RSI
Open Time,,,,,,,,,,,,,
2025-01-01 00:00:00,93576.00,93610.93,93537.50,93610.93,8.21827,2631,BTC,93609.385893,93592.819683,14.866165,11.651515,3.214650,53.112346
2025-01-01 00:01:00,93610.93,93652.00,93606.20,93652.00,12.14029,1273,BTC,93617.908714,93598.199712,17.535277,12.828267,4.707010,58.113287
2025-01-01 00:02:00,93652.00,93702.15,93635.98,93702.15,11.60597,1095,BTC,93634.756972,93607.649738,23.427200,14.948054,8.479146,63.265547
2025-01-01 00:03:00,93702.14,93702.15,93654.48,93677.98,8.72958,1461,BTC,93643.401577,93614.043398,25.848311,17.128105,8.720205,59.468887
2025-01-01 00:04:00,93677.98,93677.99,93659.92,93661.20,5.24749,988,BTC,93646.961262,93618.330362,26.112049,18.924894,7.187155,56.915228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28 15:56:00,83879.52,83912.49,83800.00,83800.01,61.60958,7231,BTC,83939.917482,83916.363430,57.714012,87.959848,-30.245836,46.766248
2025-02-28 15:57:00,83800.01,83888.29,83768.53,83849.54,46.47085,7604,BTC,83921.841986,83910.288573,46.117191,79.591316,-33.474126,48.929368
2025-02-28 15:58:00,83849.54,83947.75,83849.54,83947.75,30.48454,6875,BTC,83927.023588,83913.694157,44.340226,72.541098,-28.200872,53.006920


In [7]:
sequence_length = 60
forecast_horizon = 1 # to predict next 15 minutes
train_size = 0.8
# Extract feature column and convert to numpy array
dataset = data['Close'].values.reshape(-1, 1)

# Store corresponding dates for later reference
dates = data.index

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Create sequences
X, y, sequence_dates = [], [], []
for i in range(len(scaled_data) - sequence_length - forecast_horizon + 1):  # Adjust to handle 5 minutes
    X.append(scaled_data[i:i+sequence_length, 0])
    y.append(scaled_data[i+sequence_length:i+sequence_length+forecast_horizon, 0])  # Next 5 minutes
    # Store the date corresponding to the last value in the target (y) sequence
    sequence_dates.append(dates[i+sequence_length+forecast_horizon-1])

X, y = np.array(X), np.array(y)

# Reshape X to 3D format expected by LSTM [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split into training and testing sets
train_size = int(len(X) * train_size)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
train_dates = sequence_dates[:train_size]
test_dates = sequence_dates[train_size:]

In [8]:
units=50
dropout=0.2
layers=1

model = Sequential()
    
# First LSTM layer
if layers == 1:
    model.add(LSTM(units=units, input_shape=(sequence_length, 1)))
else:
    model.add(LSTM(units=units, return_sequences=True, input_shape=(sequence_length, 1)))
    model.add(Dropout(dropout))
    
    # Additional LSTM layers
    for i in range(layers-2):
        model.add(LSTM(units=units, return_sequences=True))
        model.add(Dropout(dropout))
    
    # Final LSTM layer
    model.add(LSTM(units=units))

model.add(Dropout(dropout))
model.add(Dense(units=forecast_horizon))  # Output layer

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

/Users/linyan/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
# Train model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

Epoch 1/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - loss: 0.0094 - val_loss: 9.7619e-05
Epoch 2/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 0.0012 - val_loss: 1.2819e-05
Epoch 3/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 4.0338e-04 - val_loss: 2.7381e-05
Epoch 4/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 1.4702e-04 - val_loss: 1.4185e-05
Epoch 5/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 1.1578e-04 - val_loss: 1.9973e-05
Epoch 6/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 1.1472e-04 - val_loss: 6.4035e-06
Epoch 7/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - loss: 1.1231e-04 - val_loss: 5.6488e-06
Epoch 8/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 1.1675e-04 - val_loss: 5.2355e-06
Epoch 9/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 1.1004e-04 - val_loss: 5.6149e-06
Epoch 10/50
1900/1900 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - loss: 1.0871e-04 - val_loss: 5.5053e-06
Epoch 11/50
1900/1900 ━━━━━━━━━━━━━━

In [10]:
#Reshape predictions for inverse transform
train_predictions = train_predictions.reshape(-1, 1)
test_predictions = test_predictions.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Inverse transform
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)
train_actual = scaler.inverse_transform(y_train)
test_actual = scaler.inverse_transform(y_test)

In [11]:
train_mse = mean_squared_error(train_actual, train_predictions)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(train_actual, train_predictions)

test_mse = mean_squared_error(test_actual, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(test_actual, test_predictions)


train_mape = np.mean(np.abs((train_actual - train_predictions) / train_actual)) * 100
test_mape = np.mean(np.abs((test_actual - test_predictions) / test_actual)) * 100

In [12]:
"""feature = 'Close' 
# Create figure
plt.figure(figsize=(16, 8))

# Plot historical data
plt.plot(data.index, data[feature], label='Actual Prices', color='blue')

# Plot predictions
plt.plot(train_dates, train_predictions, label='Training Predictions', color='green', alpha=0.8)
plt.plot(test_dates, test_predictions, label='Testing Predictions', color='red', alpha=0.8)

# Format x-axis as dates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.gcf().autofmt_xdate()  # Rotate date labels

# Add grid, labels and legend
plt.grid(True, alpha=0.3)
plt.title(f'Cryptocurrency {feature} Price Prediction', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel(f'{feature} Price (USD)', fontsize=14)
plt.legend(fontsize=12)

plt.tight_layout()
plt.show() """

"feature = 'Close' \n# Create figure\nplt.figure(figsize=(16, 8))\n\n# Plot historical data\nplt.plot(data.index, data[feature], label='Actual Prices', color='blue')\n\n# Plot predictions\nplt.plot(train_dates, train_predictions, label='Training Predictions', color='green', alpha=0.8)\nplt.plot(test_dates, test_predictions, label='Testing Predictions', color='red', alpha=0.8)\n\n# Format x-axis as dates\nplt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))\nplt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))\nplt.gcf().autofmt_xdate()  # Rotate date labels\n\n# Add grid, labels and legend\nplt.grid(True, alpha=0.3)\nplt.title(f'Cryptocurrency {feature} Price Prediction', fontsize=16)\nplt.xlabel('Date', fontsize=14)\nplt.ylabel(f'{feature} Price (USD)', fontsize=14)\nplt.legend(fontsize=12)\n\nplt.tight_layout()\nplt.show() "

In [13]:
print(train_mse)
print(train_rmse)
print(train_mae)

8990.187259218916
94.81659801542615
66.08498807580409


In [14]:
print(test_mse)
print(test_rmse)
print(test_mae)

9701.44880099229
98.49593291599552
64.41285525244311


In [15]:
print(train_mape)
print(test_mape)

0.06584506515942253
0.07246864863073423


Retrain by adding buy/sell signals

In [16]:
signals = []
    
for i in range(0, len(data)):
    # Buy signal: EMA(9) crosses above EMA(21)
    if data['EMA_9'][i] > data['EMA_21'][i] and data['EMA_9'][i-1] <= data['EMA_21'][i-1] and data['MACD'][i] > data['MACD_Signal'][i] and data['MACD_Hist'][i] > 0:
           
        signals.append('1') # Buy
    # Sell signal: EMA(9) crosses below EMA(21)
    elif data['EMA_9'][i] < data['EMA_21'][i] and data['EMA_9'][i-1] >= data['EMA_21'][i-1] and data['MACD'][i] < data['MACD_Signal'][i] and data['MACD_Hist'][i] < 0:
        signals.append('-1') # Sell
    else: 
        signals.append('0') # Hold

/var/folders/9d/bxxrjx5d1qg5y0x87t0hp84w0000gn/T/ipykernel_24903/1258329814.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if data['EMA_9'][i] > data['EMA_21'][i] and data['EMA_9'][i-1] <= data['EMA_21'][i-1] and data['MACD'][i] > data['MACD_Signal'][i] and data['MACD_Hist'][i] > 0:
/var/folders/9d/bxxrjx5d1qg5y0x87t0hp84w0000gn/T/ipykernel_24903/1258329814.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif data['EMA_9'][i] < data['EMA_21'][i] and data['EMA_9'][i-1] >= data['EMA_21'][i-1] and data['MACD'][i] < data['MACD_Signal'][i] and data['MACD_Hist'][i] < 0:


In [17]:
data['Buy or Sell'] = signals

In [18]:
data

,Open,High,Low,Close,Volume,Number of Trades,Coin,EMA_9,EMA_21,MACD,MACD_Signal,MACD_Hist,RSI,Buy or Sell
Open Time,,,,,,,,,,,,,,
2025-01-01 00:00:00,93576.00,93610.93,93537.50,93610.93,8.21827,2631,BTC,93609.385893,93592.819683,14.866165,11.651515,3.214650,53.112346,0
2025-01-01 00:01:00,93610.93,93652.00,93606.20,93652.00,12.14029,1273,BTC,93617.908714,93598.199712,17.535277,12.828267,4.707010,58.113287,0
2025-01-01 00:02:00,93652.00,93702.15,93635.98,93702.15,11.60597,1095,BTC,93634.756972,93607.649738,23.427200,14.948054,8.479146,63.265547,0
2025-01-01 00:03:00,93702.14,93702.15,93654.48,93677.98,8.72958,1461,BTC,93643.401577,93614.043398,25.848311,17.128105,8.720205,59.468887,0
2025-01-01 00:04:00,93677.98,93677.99,93659.92,93661.20,5.24749,988,BTC,93646.961262,93618.330362,26.112049,18.924894,7.187155,56.915228,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28 15:56:00,83879.52,83912.49,83800.00,83800.01,61.60958,7231,BTC,83939.917482,83916.363430,57.714012,87.959848,-30.245836,46.766248,0
2025-02-28 15:57:00,83800.01,83888.29,83768.53,83849.54,46.47085,7604,BTC,83921.841986,83910.288573,46.117191,79.591316,-33.474126,48.929368,0
2025-02-28 15:58:00,83849.54,83947.75,83849.54,83947.75,30.48454,6875,BTC,83927.023588,83913.694157,44.340226,72.541098,-28.200872,53.006920,0


Use the above model to test out predicting the next 5 minutes of prices, and also continue to calculate the buy sell or hold signal. 

If we have a BUY signal, we can consider doing a portfolio reallocation.

In [19]:
latest_prices = data.iloc[-1]

In [20]:
data['Close'].tolist()

[93610.93,
 93652.0,
 93702.15,
 93677.98,
 93661.2,
 93678.01,
 93632.05,
 93612.01,
 93611.84,
 93607.1,
 93538.61,
 93520.54,
 93519.38,
 93544.49,
 93656.18,
 93702.27,
 93768.02,
 93827.31,
 93824.52,
 93796.35,
 93866.0,
 93800.0,
 93733.5,
 93748.42,
 93740.0,
 93734.5,
 93733.63,
 93727.31,
 93732.03,
 93761.9,
 93798.21,
 93842.33,
 93931.99,
 93892.3,
 93882.92,
 93884.27,
 93861.48,
 93840.92,
 93924.6,
 93909.03,
 93933.18,
 93892.0,
 93893.17,
 93910.01,
 93885.01,
 93864.45,
 93890.26,
 93959.99,
 93977.97,
 93977.97,
 94087.63,
 94170.0,
 94158.21,
 94199.99,
 94260.0,
 94216.0,
 94224.63,
 94330.61,
 94477.83,
 94401.14,
 94280.48,
 94220.01,
 94187.5,
 94142.94,
 94118.41,
 93918.77,
 93935.46,
 93969.71,
 93983.02,
 94061.41,
 94079.65,
 94096.25,
 94152.16,
 94146.4,
 94153.05,
 94068.45,
 94011.93,
 93982.21,
 93978.95,
 93990.24,
 93977.8,
 93977.8,
 93977.8,
 93864.94,
 93898.13,
 93899.7,
 93876.34,
 93877.34,
 93848.37,
 93825.86,
 93899.99,
 93920.48,
 93942.65

In [21]:
"""
for_prediction = data.iloc[-sequence_length:] # take the last 5 rows
# init new values

current_ema_9 = data.iloc[-1]['EMA_9']
current_ema_21 = data.iloc[-1]['EMA_21']
current_macd = data.iloc[-1]['MACD']
current_macd_sig = data.iloc[-1]['MACD_Signal']
current_macd_hist = data.iloc[-1]['MACD_Hist']

next_prices = model.predict(scaler.fit_transform(for_prediction))
next_prices = scaler.inverse_transform(next_prices)
all_current_prices = data['Close']
predicted_signals = []

for x in next_prices:
    new_price = x
    all_current_prices.append(new_price)
    #recalculate the EMA & MACD
    new_ema_9 = talib.EMA(all_current_prices, timeperiod=9)
    new_ema_21 = talib.EMA(all_current_prices, timeperiod=21)
    new_macd, new_macd_sig, new_macd_hist = talib.MACD(all_current_prices, fastperiod=12, slowperiod=26, signalperiod=9)

    # Buy signal: EMA(9) crosses above EMA(21)
    if new_ema_9 > new_ema_21 and current_ema_9 <= current_ema_21 and new_macd > new_macd_sig and new_macd_hist > 0:
        predicted_signals.append('1') # Buy
    # Sell signal: EMA(9) crosses below EMA(21)
    elif new_ema_9 < new_ema_21  and current_ema_9 >= current_ema_21 and new_macd < new_macd_sig and new_macd_hist < 0:
        predicted_signals.append('-1') # Sell
    else: 
        predicted_signals.append('0') # Hold

    # Replace values for next loop
    current_ema_9 = new_ema_9
    current_ema_21 = new_ema_21
    current_macd = new_macd
    current_macd_sig = new_macd_sig
    current_macd_hist = new_macd_hist

"""

ValueError: could not convert string to float: 'BTC'